## NASA-POWER to LARS-WG


In [1]:
import datetime as dt
import numpy as np 
import os
import requests
import pandas as pd


class NASA_POWER:
    ranges = {"LAT": (-90., 90.),
          "LON": (-180., 180.),
          "ELEV": (-300, 6000),
          "IRRAD": (0., 40e6),
          "TMIN": (-50., 60.),
          "TMAX": (-50., 60.),
          "VAP": (0.06, 199.3),  # hPa, computed as sat. vapour pressure at -50, 60 Celsius
          "RAIN": (0, 25),
          "E0": (0., 2.5),
          "ES0": (0., 2.5),
          "ET0": (0., 2.5),
          "WIND": (0., 100.),
          "SNOWDEPTH": (0., 250.),
          "TEMP": (-50., 60.),
          "TMINRA": (-50., 60.)}

    def __init__(self, latitude, longitude):
        if latitude < -90 or latitude > 90:
            msg = "Latitude should be between -90 and 90 degrees."
            raise ValueError(msg)
        if longitude < -180 or longitude > 180:
            msg = "Longitude should be between -180 and 180 degrees."
            raise ValueError(msg)

        self.MJ_to_J = lambda x: x * 1e6
        self.mm_to_cm = lambda x: x / 10.
        self.tdew_to_hpa = lambda x: ea_from_tdew(x) * 10.
        self.to_date = lambda d: d.date()
        self.HTTP_OK = 200
        self.angstA = 0.29
        self.angstB = 0.49

        self.latitude = latitude
        self.longitude = longitude

        self.power_variables = ["TOA_SW_DWN", "ALLSKY_SFC_SW_DWN", "T2M", "T2M_MIN",
                            "T2M_MAX", "T2MDEW", "WS2M", "PRECTOTCORR", 'RH2M']
        self._get_and_process_NASAPower(latitude, longitude)

    def _get_and_process_NASAPower(self, latitude, longitude):
            """Handles the retrieval and processing of the NASA Power data
            """
            powerdata = self._query_NASAPower_server(latitude, longitude)
            if not powerdata:
                msg = "Failure retrieving POWER data from server. This can be a connection problem with " \
                    "the NASA POWER server, retry again later."
                raise RuntimeError(msg)

            # Store the informational header then parse variables
            self.description = [powerdata["header"]["title"]]
            self.elevation = float(powerdata["geometry"]["coordinates"][2])
            
            
            df_power = self._process_POWER_records(powerdata)
#             self.angstA, self.angstB = self._estimate_AngstAB(df_power)
            df_monica = self._POWER_to_PCSE(df_power)
            self.df_monica = df_monica
            return df_monica
    def _query_NASAPower_server(self, latitude, longitude):
        start_date = dt.date(1985,1,1)
        end_date = dt.date(2023,12,31)
        # build URL for retrieving data, using new NASA POWER api
        server = "https://power.larc.nasa.gov/api/temporal/daily/point"
        payload = {"request": "execute",
                    "parameters": ",".join(self.power_variables),
                    "latitude": latitude,
                    "longitude": longitude,
                    "start": start_date.strftime("%Y%m%d"),
                    "end": end_date.strftime("%Y%m%d"),
                    "community": "AG",
                    "format": "JSON",
                    "user": "anonymous"
                    }
        req = requests.get(server, params=payload)

        if req.status_code != self.HTTP_OK:
            msg = ("Failed retrieving POWER data, server returned HTTP " +
                    "code: %i on following URL %s") % (req.status_code, req.url)
            raise ValueError(msg)

        return req.json()

    def _process_POWER_records(self, powerdata):
        """Process the meteorological records returned by NASA POWER
        """


        fill_value = float(powerdata["header"]["fill_value"])

        df_power = {}
        for varname in self.power_variables:
            s = pd.Series(powerdata["properties"]["parameter"][varname])
            s[s == fill_value] = np.NaN
            df_power[varname] = s
        df_power = pd.DataFrame(df_power)
        df_power["DAY"] = pd.to_datetime(df_power.index, format="%Y%m%d")

        # find all rows with one or more missing values (NaN)
        ix = df_power.isnull().any(axis=1)
        # Get all rows without missing values
        df_power = df_power[~ix]

        return df_power
    
    def _POWER_to_PCSE(self, df_power):

            # Convert POWER data to a dataframe with PCSE compatible inputs
            df_pcse = pd.DataFrame({"date": df_power.DAY.apply(self.to_date),
                                    "MIN": df_power.T2M_MIN,
                                    "MAX": df_power.T2M_MAX,
                                    "RAD": df_power.ALLSKY_SFC_SW_DWN,
                                    "RAIN": df_power.PRECTOTCORR})
#             df_pcse.loc[:, 'de-date'] = df_pcse.loc[:, 'de-date'].apply(lambda x: csvdate_to_date(x, '%Y-%m-%d'))
            self.df_pcse = df_pcse.reset_index(drop=True)
            return df_pcse
        

In [2]:

nasa_weather = NASA_POWER(latitude = 51.5, longitude = 37.1)
df_weather = nasa_weather.df_pcse
df_weather.loc[:,'date'] = pd.to_datetime(df_weather.loc[:,'date'])
r = pd.date_range(start=df_weather.loc[:,'date'].min(), end=df_weather.loc[:,'date'].max())
full_range_weather = df_weather.set_index('date').reindex(r).rename_axis('date').reset_index()
filled_weather = full_range_weather.fillna(method='ffill', axis=0)      
filled_weather['year'] = filled_weather['date'].dt.year
filled_weather['month'] = filled_weather['date'].dt.month
filled_weather['day'] = filled_weather['date'].dt.day


filled_weather[['year', 'month', 'day', 'MIN', 'MAX', 'RAIN','RAD']].to_csv('LARS_KSHEN.dat', 
                                                                            sep='\t', index=False, 
                                                                           header=False)

### Read LARS-WG data

In [6]:
import os
import pandas as pd

In [7]:
folder = '../src/LARS-WG/LARS_KSHEN/'
files = os.listdir(folder)

In [19]:
folder = '../src/LARS-WG/LARS_KSHEN/'
files = os.listdir(folder)
columns = ["YEAR", "JDAY", "MIN", "MAX", "RAIN","RAD"]
for model in ["HadGEM3-GC31-LL", "CNRM-CM6-1"]:
    for scenario in ['ssp126', 'ssp585']:
        for period in ['2081-2100']:
            fname = f'LARS_KSHEN_{model}[LARS_KSHEN,{scenario},{period}]WG.dat'
            target = os.path.join(folder, fname)
            data = pd.read_csv(target, sep='\t', index_col=False, header=None, names=columns )
            data['date'] = pd.to_datetime(data['JDAY'].apply(lambda x: '2081-'+str(x)), format='%Y-%j')

In [18]:
data['date'] = pd.to_datetime(data['JDAY'].apply(lambda x: '2081-'+str(x)), format='%Y-%j')

## End